# Million Song Data Set & Complimentary Datasets Wrangling

## **Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import h5py
import csv
import zipfile
from urllib.request import urlretrieve
from sqlalchemy import create_engine

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Part 1. MSD Files Exploration

> ### <font color =red >_Single File Structure Exploration_</font>

In [2]:
#retrieve single .h5 file for exploration.

%time

url = 'http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/TRAXLZU12903D05F94.h5'

urlretrieve(url, 'TRAXLZU12903D05F94.h5')

example_complete_file = h5py.File('TRAXLZU12903D05F94.h5', 'r')

print(type(example_complete_file))

CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 27.2 µs
<class 'h5py._hl.files.File'>


In [3]:
#This is an example of a complete MSD song file. 
#These files include all information available regarding the MSD. 
#This includes tags, in-depth song analysis and artist similarity analysis. 
#Each .h5 file represents one song. In total, they amount to 1M song files.
    
print()
print('File Groups = 3')
print('-------------------------------------------------')

for key in example_complete_file.keys():
    print(key)

print()
print('Data within the ANALYSIS group')
print('-------------------------------------------------')
    
for key in example_complete_file['analysis']:
    print(key)
    
print()
print('Data within the METADATA group')
print('-------------------------------------------------')

for key in example_complete_file['metadata']:
    print(key)

print()
print('Data within the MUSICBRAINZ group')
print('-------------------------------------------------')

for key in example_complete_file['musicbrainz']:
    print(key)



File Groups = 3
-------------------------------------------------
analysis
metadata
musicbrainz

Data within the ANALYSIS group
-------------------------------------------------
bars_confidence
bars_start
beats_confidence
beats_start
sections_confidence
sections_start
segments_confidence
segments_loudness_max
segments_loudness_max_time
segments_loudness_start
segments_pitches
segments_start
segments_timbre
songs
tatums_confidence
tatums_start

Data within the METADATA group
-------------------------------------------------
artist_terms
artist_terms_freq
artist_terms_weight
similar_artists
songs

Data within the MUSICBRAINZ group
-------------------------------------------------
artist_mbtags
artist_mbtags_count
songs


> ### <font color =red >_Metadata Aggregate File Exploration_</font>

In [4]:
#Retrieve msd aggregate metadata file. The file is 300MB.
#These files include only song metadata information. 
#These files does not includes tags, in-depth song analysis and artist similarity analysis. 
#All the information is contained in one .h5 file, which makes it much smaller than the 
#full dataset of 1M separate files.

%time

url = 'http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/msd_summary_file.h5'

urlretrieve(url, 'msd_summary_file.h5')

metadata_summary_file = h5py.File('msd_summary_file.h5', 'r')

print(type(metadata_summary_file))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs
<class 'h5py._hl.files.File'>


In [5]:
%time

#local copy path: metadata_summary_file = h5py.File('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD Additional Files/msd_summary_file.h5','r')

#see groups and keys in aggregate metadata .h5 file.

print()
print('File Groups = 3')
print('-------------------------------------------------')

for key in metadata_summary_file.keys():
    print(key)


print()
print('Keys within the ANALYSIS group')
print('-------------------------------------------------')
    
for key in metadata_summary_file['analysis']:
    print(key)
    
print()
print('Keys within the METADATA group')
print('-------------------------------------------------')

for key in metadata_summary_file['metadata']:
    print(key)

print()
print('Keys within the MUSICBRAINZ group')
print('-------------------------------------------------')

for key in metadata_summary_file['musicbrainz']:
    print(key)
    
#this file was created by the using an iterator 
#that aggregates the metadata files from the
#individual .h5 song files. 
#the code can be found here:
#https://github.com/tbertinmahieux/MSongsDB/blob/master/PythonSrc/create_aggregate_file.py
#as a result the arrays are ordered by song 
#(example song 1 of metadata group = song 1 of analysis group)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 30 µs

File Groups = 3
-------------------------------------------------
analysis
metadata
musicbrainz

Keys within the ANALYSIS group
-------------------------------------------------
songs

Keys within the METADATA group
-------------------------------------------------
songs

Keys within the MUSICBRAINZ group
-------------------------------------------------
songs


## Part 2. Upload Million Song Dataset Data

### Approach 1 | *Upload Metadata Song Files & Tags Database*

> ### <font color =red >_MSD Metadata Song Files Upload_</font>

In [6]:
#Upload song metadata informaton into a dataframe. See available columns in the dataset.

%time

song_metadata = pd.DataFrame(metadata_summary_file['metadata']['songs'].value)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [7]:
#See song metadata dataframe information. There are some missing values in:
#artist_familiarity, artist_latitude,artist_longitude,and song_hotttnesss columns. 

song_metadata.info()

song_metadata.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
analyzer_version       1000000 non-null object
artist_7digitalid      1000000 non-null int32
artist_familiarity     999815 non-null float64
artist_hotttnesss      999988 non-null float64
artist_id              1000000 non-null object
artist_latitude        357492 non-null float64
artist_location        1000000 non-null object
artist_longitude       357492 non-null float64
artist_mbid            1000000 non-null object
artist_name            1000000 non-null object
artist_playmeid        1000000 non-null int32
genre                  1000000 non-null object
idx_artist_terms       1000000 non-null int32
idx_similar_artists    1000000 non-null int32
release                1000000 non-null object
release_7digitalid     1000000 non-null int32
song_hotttnesss        581965 non-null float64
song_id                1000000 non-null object
title                  1000000 non-null object


,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
0,b'',4069,0.649822,0.394032,b'ARYZTJS1187B98C555',NaN,b'',NaN,b'357ff05d-848a-44cf-b608-cb34b5701ae5',b'Faster Pussy cat',44895,b'',0,0,b'Monster Ballads X-Mas',633681,0.542899,b'SOQMMHC12AB0180CB8',b'Silent Night',7032331
1,b'',113480,0.439604,0.356992,b'ARMVN3U1187FB3A1EB',NaN,b'',NaN,b'8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9',b'Karkkiautomaatti',-1,b'',0,0,b'Karkuteill\xc3\xa4',145266,0.299877,b'SOVFVAK12A8C1350D9',b'Tanssi vaan',1514808
2,b'',63531,0.643681,0.437504,b'ARGEKB01187FB50750',55.8578,"b'Glasgow, Scotland'",-4.24251,b'3d403d44-36ce-465c-ad43-ae877e65adc4',b'Hudson Mohawke',-1,b'',0,0,b'Butter',625706,0.617871,b'SOGTUKN12AB017F4F1',b'No One Could Ever',6945353
3,b'',65051,0.448501,0.372349,b'ARNWYLR1187B9B2F9C',NaN,b'',NaN,b'12be7648-7094-495f-90e6-df4189d68615',b'Yerba Brava',34000,b'',0,0,b'De Culo',199368,NaN,b'SOBNYVR12A8C13558C',b'Si Vos Quer\xc3\xa9s',2168257
4,b'',158279,0.000000,0.000000,b'AREQDTE1269FB37231',NaN,b'',NaN,b'',b'Der Mystic',-1,b'',0,0,b'Rene Ablaze Presents Winter Sessions',209038,NaN,b'SOHSBXH12A8C13B0DF',b'Tangle Of Aspens',2264873


In [8]:
#Convert objects into strings. This is to remove unecesary strings below.

%time

song_metadata['analyzer_version'] = song_metadata['analyzer_version'].astype(str)

song_metadata['artist_id'] = song_metadata['artist_id'].astype(str)

song_metadata['artist_location'] = song_metadata['artist_location'].astype(str)

song_metadata['artist_mbid'] = song_metadata['artist_mbid'].astype(str)

song_metadata['artist_name'] = song_metadata['artist_name'].astype(str)

song_metadata['genre'] = song_metadata['genre'].astype(str)

song_metadata['release'] = song_metadata['release'].astype(str)

song_metadata['song_id'] = song_metadata['song_id'].astype(str)

song_metadata['title'] = song_metadata['title'].astype(str)


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 23.8 µs


In [9]:
#Remove 'b' string from object columns. This is to make it consistent and easier to join
#with other information later if needed.

%time

song_metadata['analyzer_version'] = song_metadata['analyzer_version'].str.lstrip("b'").str.rstrip("'")

song_metadata['artist_id'] = song_metadata['artist_id'].str.lstrip("b'").str.rstrip("'")

song_metadata['artist_location'] = song_metadata['artist_location'].str.lstrip("b'").str.rstrip("'")

song_metadata['artist_mbid'] = song_metadata['artist_mbid'].str.lstrip("b'").str.rstrip("'")

song_metadata['artist_name'] = song_metadata['artist_name'].str.lstrip("b'").str.rstrip("'")

song_metadata['genre'] = song_metadata['genre'].str.lstrip("b'").str.rstrip("'")

song_metadata['release'] = song_metadata['release'].str.lstrip("b'").str.rstrip("'")

song_metadata['song_id'] = song_metadata['song_id'].str.lstrip("b'").str.rstrip("'")

song_metadata['title'] = song_metadata['title'].str.lstrip("b'").str.rstrip("'")


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [10]:
#See head of the song metadata dataframe

song_metadata.head()

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
0,,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,44895,,0,0,Monster Ballads X-Mas,633681,0.542899,SOQMMHC12AB0180CB8,Silent Night,7032331
1,,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,-1,,0,0,Karkuteill\xc3\xa4,145266,0.299877,SOVFVAK12A8C1350D9,Tanssi vaan,1514808
2,,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,-1,,0,0,Butter,625706,0.617871,SOGTUKN12AB017F4F1,No One Could Ever,6945353
3,,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,34000,,0,0,De Culo,199368,NaN,SOBNYVR12A8C13558C,Si Vos Quer\xc3\xa9s,2168257
4,,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,,NaN,,Der Mystic,-1,,0,0,Rene Ablaze Presents Winter Sessions,209038,NaN,SOHSBXH12A8C13B0DF,Tangle Of Aspens,2264873


> ### <font color =red >_MSD Analysis Metadata File Upload_</font>

In [11]:
#Upload analysis metadata informaton into a dataframe. See available columns in the dataset. 

%time

analysis_metadata = pd.DataFrame(metadata_summary_file['analysis']['songs'].value)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs


In [12]:
#See song metadata dataframe information. It seems that there are no missing values.

analysis_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 31 columns):
analysis_sample_rate              1000000 non-null int32
audio_md5                         1000000 non-null object
danceability                      1000000 non-null float64
duration                          1000000 non-null float64
end_of_fade_in                    1000000 non-null float64
energy                            1000000 non-null float64
idx_bars_confidence               1000000 non-null int32
idx_bars_start                    1000000 non-null int32
idx_beats_confidence              1000000 non-null int32
idx_beats_start                   1000000 non-null int32
idx_sections_confidence           1000000 non-null int32
idx_sections_start                1000000 non-null int32
idx_segments_confidence           1000000 non-null int32
idx_segments_loudness_max         1000000 non-null int32
idx_segments_loudness_max_time    1000000 non-null int32
idx_segments_loudness_st

In [13]:
#Convert objects into strings. This is to remove unecesary strings below.

analysis_metadata['audio_md5'] = analysis_metadata['audio_md5'].astype(str)

analysis_metadata['track_id'] = analysis_metadata['track_id'].astype(str)

In [14]:
#Remove 'b' string from object columns. This is to make it consistent and easier to join
#with other information later if needed.

analysis_metadata['audio_md5'] = analysis_metadata['audio_md5'].str.lstrip("b'").str.rstrip("'")

analysis_metadata['track_id'] = analysis_metadata['track_id'].str.lstrip("b'").str.rstrip("'")


In [15]:
#See top of the analysis metadata

analysis_metadata.head()

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,aee9820911781c734e7694c5432990ca,0.0,252.05506,2.049,0.0,0,0,0,0,...,10,0.777,-4.829,0,0.688,236.635,87.002,4,0.940,TRMMMYQ128F932D901
1,22050,ed222d07c83bac7689d52753610a513a,0.0,156.55138,0.258,0.0,0,0,0,0,...,9,0.808,-10.555,1,0.355,148.660,150.778,1,0.000,TRMMMKD128F425225D
2,22050,96c7104889a128fef84fa469d60e380c,0.0,138.97098,0.000,0.0,0,0,0,0,...,7,0.418,-2.060,1,0.566,138.971,177.768,4,0.446,TRMMMRX128F93187D9
3,22050,0f7da84b6b583e3846c7e022fb3a92a2,0.0,145.05751,0.000,0.0,0,0,0,0,...,7,0.125,-4.654,1,0.451,138.687,87.433,4,0.000,TRMMMCH128F425532C
4,22050,228dd6392ad8001b0281f533f34c72fd,0.0,514.29832,0.000,0.0,0,0,0,0,...,5,0.097,-7.806,0,0.290,506.717,140.035,4,0.315,TRMMMWA128F426B589


> ### <font color =red >_MSD Musicbrainz Metadata File Upload_</font>

In [16]:
#Upload musicbrainz metadata informaton into a dataframe. See available columns in the dataset.  

%time

musicbrainz_metadata = pd.DataFrame(metadata_summary_file['musicbrainz']['songs'].value)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10 µs


In [17]:
#See song metadata dataframe information. It seems that there are no missing values.

musicbrainz_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
idx_artist_mbtags    1000000 non-null int32
year                 1000000 non-null int32
dtypes: int32(2)
memory usage: 7.6 MB


In [18]:
#See top of the musicbrainz metadata. This dataset has year of song release info.

#Zeroes indicate missing values.

musicbrainz_metadata.head(10)

,idx_artist_mbtags,year
0,0,2003
1,0,1995
2,0,2006
3,0,2003
4,0,0
5,0,0
6,0,0
7,0,1993
8,0,0
9,0,0


In [19]:
#Zeros represent missing values for the year column. Fill with NaN. 

musicbrainz_metadata['year'] = musicbrainz_metadata['year'].replace(0,np.nan)

musicbrainz_metadata.info() # 484,424 missing values created

musicbrainz_metadata.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
idx_artist_mbtags    1000000 non-null int32
year                 515576 non-null float64
dtypes: float64(1), int32(1)
memory usage: 11.4 MB


,idx_artist_mbtags,year
0,0,2003.0
1,0,1995.0
2,0,2006.0
3,0,2003.0
4,0,NaN
5,0,NaN
6,0,NaN
7,0,1993.0
8,0,NaN
9,0,NaN


> ### <font color =red >_MSD Song Tags Database Upload_</font>

In [20]:
#Upload SQLite database containing artist tags.
#The database is saved locally. It came as part of folder A of the complete MSD files. 
#Due to the large file size, the rsync protocol from the terminal must be used to obtain the data.
#rsync -avzuP publicdata.opensciencedatacloud.org::ark:/31807/osdc-c1c763e4/remotefile /path/to/local_copy
#More info here: https://www.opensciencedatacloud.org/publicdata/million-song-dataset/

engine = create_engine('sqlite:////Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD Additional Files/artist_term.db')

table_names = engine.table_names()

table_names

['artist_mbtag', 'artist_term', 'artists', 'mbtags', 'terms']

In [21]:
#Query artist_mbtag table and pass to a dataframe

con = engine.connect()

rs = con.execute("SELECT * FROM artist_mbtag")

artist_mbtag = pd.DataFrame(rs.fetchall(),columns=['artist_id','mbtag'])

con.close()

artist_mbtag.head()


,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock
3,AR006821187FB5192B,bass
4,AR00A6H1187FB5402A,detroit


In [22]:
#Query artist_term table and pass to a dataframe

con = engine.connect()

rs = con.execute("SELECT * FROM artist_term")

artist_terms = pd.DataFrame(rs.fetchall(),columns=['artist_id','terms'])

con.close()

artist_terms.head()

,artist_id,terms
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz
3,AR002UA1187B9A637D,oi
4,AR002UA1187B9A637D,space rock


In [23]:
#Merge term and mbtags dataframes

artist_tags = artist_mbtag.merge(artist_terms,on='artist_id',how='outer')

artist_tags

artist_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621124 entries, 0 to 1621123
Data columns (total 3 columns):
artist_id    1621124 non-null object
mbtag        798145 non-null object
terms        1621118 non-null object
dtypes: object(3)
memory usage: 49.5+ MB


In [24]:
#Null values were created in the merge. Fill these with NaN

artist_tags = artist_tags.fillna('NaN')

artist_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621124 entries, 0 to 1621123
Data columns (total 3 columns):
artist_id    1621124 non-null object
mbtag        1621124 non-null object
terms        1621124 non-null object
dtypes: object(3)
memory usage: 49.5+ MB


In [25]:
#See artist tags df

artist_tags.head()


,artist_id,mbtag,terms
0,AR002UA1187B9A637D,uk,garage rock
1,AR002UA1187B9A637D,uk,country rock
2,AR002UA1187B9A637D,uk,free jazz
3,AR002UA1187B9A637D,uk,oi
4,AR002UA1187B9A637D,uk,space rock


# Approach 2 | *Create Summary File Through Iteration*

*This is an alternative to using the summary metadata file and the SQLite database.*

In [26]:
#Function that grabs file list from a directory. Looks through all folders within the directory.

def get_file_list(basedir,ext='.h5'):
    """
    From a root directory, go through all subdirectories
    and find all files with the given extension.
    Return all absolute paths in a list.
    """
    allfiles = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files :
            allfiles.append( os.path.abspath(f) )
    return allfiles

In [27]:
%time

#Due to the large file size, the rsync protocol from the terminal must be used to obtain the data.
#rsync -avzuP publicdata.opensciencedatacloud.org::ark:/31807/osdc-c1c763e4/remotefile /path/to/local_copy
#More info here: https://www.opensciencedatacloud.org/publicdata/million-song-dataset/

basedir = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/MSD Complete Dataset'

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 15 µs


In [28]:
#Create list of files

%time

list_of_files = get_file_list(basedir,ext='.h5')

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.9 µs


In [29]:
# Create the list of test files

test_list_of_files = glob.glob('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/Test/Test1/*.h5')

In [30]:
%time

#Iterate over the list of files and put into df.

array_append = []
group='metadata'
key='songs'

for file_name in test_list_of_files:
    data = h5py.File(file_name, 'r')
    array = data[group][key].value
    array_append.append(array)
    df= pd.DataFrame(array_append)

df

#This takes a while to run with a large set of files. Here only 10 are showed.
#I would use the list_of_files to run over all MSD files.
#If I want to use this approach I need to unpack the list into columns.

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


,0
0,"[b'', 63586, 0.6253352989877244, 0.42263007412..."
1,"[b'', 162979, 0.8430415025930272, 0.5683041121..."
2,"[b'', 436731, 0.5233810603031716, 0.3502998449..."
3,"[b'', 7552, 0.5882619172748459, 0.332287249059..."
4,"[b'', 260085, 0.3007892063841431, 0.0, b'ARUAP..."
5,"[b'', 293614, 0.37075813800896407, 0.367097301..."
6,"[b'', 326050, 0.5565712223702348, 0.4362577592..."
7,"[b'', 217429, 0.9061988755957036, 0.5635331069..."
8,"[b'', 296278, 0.49941298004250484, 0.0, b'AR7G..."
9,"[b'', 10239, 0.573039790503387, 0.437866012408..."


I decided not to use approach 2. Approach 1 gives the data I needed more efficiently since the iteration demostrated above has already been done.

## Part 3. Upload User Listening Data

In [31]:
#Upload user listening data (500MB). 
#Can be found here: 
#http://labrosa.ee.columbia.edu/millionsong/sites/default/files/challenge/train_triplets.txt.zip

%time

url = 'http://labrosa.ee.columbia.edu/millionsong/sites/default/files/challenge/train_triplets.txt.zip'

urlretrieve(url, 'train_triplets.txt.zip')
            
#unzip file
            
import zipfile
zip_ref = zipfile.ZipFile('train_triplets.txt.zip', 'r')
users_listening = zip_ref.extractall('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data')
zip_ref.close()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


In [32]:
%time

#Loading File Testing
#10K rows: 19.4ms, walltime: 24.3ms
#50K rows: 67.5ms, walltime: 378ms
#100K rows: 127ms, walltime: 507ms
#1M rows: 1.11s, walltime: 1.58s
#10M rows: 9.65s, walltime: 11.2s
#20M rows: 19s, walltime: 22.1s
#40M rows: 39.8s, walltime: 49.3s
#Full dataset 48.8M rows: 49.2s, walltime: 1m 6s
#No errors.

users_path = '/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/Taste_Profile_Subset.txt'

users_listening_data = pd.read_csv(users_path, header = None, 
                                   names = ['user_id','song_id','play_count'],
                                   sep = '\s+')

users_listening_data.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


,user_id,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [33]:
#This adds a column with value 1 to indicate the user played the song.

users_listening_data['play'] = 1

users_listening_data.info()

users_listening_data.head()

#No missing values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48373586 entries, 0 to 48373585
Data columns (total 4 columns):
user_id       object
song_id       object
play_count    int64
play          int64
dtypes: int64(2), object(2)
memory usage: 1.4+ GB


,user_id,song_id,play_count,play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,1


## Part 4. Upload Known Song Mismatches

> #### <font color =red >_The purpose of uploading this is to clean MSD later on to remove innacurate entries that can affect model fitting later on._</font>

In [34]:
%time 

#Upload known song mismatches. 
#More information of mismatch issue can be found here:
#https://labrosa.ee.columbia.edu/millionsong/blog/12-2-12-fixing-matching-errors

url = 'http://labrosa.ee.columbia.edu/millionsong/sites/default/files/tasteprofile/sid_mismatches.txt'

urlretrieve(url, 'sid_mismatches.txt')

txt_file_mis = r'sid_mismatches.txt'

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 24.1 µs


In [35]:
#Read mistmatches txt files as a dataframe. It reads all the data as one column

msd_mismatch_data = pd.read_csv(txt_file_mis,sep='delimeter',
                                 header=None, engine = 'python',names = ['mismatches'])

msd_mismatch_data.head()

,mismatches
0,ERROR: <SOUMNSI12AB0182807 TRMMGKQ128F9325E10>...
1,ERROR: <SOCMRBE12AB018C546 TRMMREB12903CEB1B1>...
2,ERROR: <SOLPHZY12AC468ABA8 TRMMBOC12903CEB46E>...
3,ERROR: <SONGHTM12A8C1374EF TRMMITP128F425D8D0>...
4,ERROR: <SONGXCA12A8C13E82E TRMMAYZ128F429ECE6>...


In [36]:
#Read mistmatches txt files as a dataframe. It reads all the data as one column

msd_mismatch_data = pd.read_csv(txt_file_mis,sep='delimeter',
                                 header=None, engine = 'python',names = ['mismatches'])

#Explit using delimeters

msd_mismatch_data = msd_mismatch_data.mismatches.str.split(':|<|>|!=',expand=True)

msd_mismatch_data.head()

,0,1,2,3,4,5,6,7
0,ERROR,,SOUMNSI12AB0182807 TRMMGKQ128F9325E10,Digital Underground - The Way We Swing,Linkwood - Whats up with the Underground,None,None,None
1,ERROR,,SOCMRBE12AB018C546 TRMMREB12903CEB1B1,Jimmy Reed - The Sun Is Shining (Digitally ...,Slim Harpo - I Got Love If You Want It,None,None,None
2,ERROR,,SOLPHZY12AC468ABA8 TRMMBOC12903CEB46E,Africa HiTech - Footstep,Marcus Worgull - Drumstern (BONUS TRACK),None,None,None
3,ERROR,,SONGHTM12A8C1374EF TRMMITP128F425D8D0,Death in Vegas - Anita Berber,Valen Hsu - Shi Yi,None,None,None
4,ERROR,,SONGXCA12A8C13E82E TRMMAYZ128F429ECE6,Grupo Exterminador - El Triunfador,I Ribelli - Lei M'Ama,None,None,None


In [37]:
#Drop blank columns and/or unneded columns

msd_mismatch_data = msd_mismatch_data.drop([0,1,5,6,7],axis = 1)

#Remane columns

msd_mismatch_data.columns = ['song_id','song_name','does_not_equal_to_song']

msd_mismatch_data.head()

,song_id,song_name,does_not_equal_to_song
0,SOUMNSI12AB0182807 TRMMGKQ128F9325E10,Digital Underground - The Way We Swing,Linkwood - Whats up with the Underground
1,SOCMRBE12AB018C546 TRMMREB12903CEB1B1,Jimmy Reed - The Sun Is Shining (Digitally ...,Slim Harpo - I Got Love If You Want It
2,SOLPHZY12AC468ABA8 TRMMBOC12903CEB46E,Africa HiTech - Footstep,Marcus Worgull - Drumstern (BONUS TRACK)
3,SONGHTM12A8C1374EF TRMMITP128F425D8D0,Death in Vegas - Anita Berber,Valen Hsu - Shi Yi
4,SONGXCA12A8C13E82E TRMMAYZ128F429ECE6,Grupo Exterminador - El Triunfador,I Ribelli - Lei M'Ama


In [38]:
#Split song_id column into song_id and track_id

msd_mismatch_data[['song_id','track_id']] = msd_mismatch_data['song_id'].str.split(expand=True)

msd_mismatch_data.info()

msd_mismatch_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19094 entries, 0 to 19093
Data columns (total 4 columns):
song_id                   19094 non-null object
song_name                 19094 non-null object
does_not_equal_to_song    19094 non-null object
track_id                  19094 non-null object
dtypes: object(4)
memory usage: 596.8+ KB


,song_id,song_name,does_not_equal_to_song,track_id
0,SOUMNSI12AB0182807,Digital Underground - The Way We Swing,Linkwood - Whats up with the Underground,TRMMGKQ128F9325E10
1,SOCMRBE12AB018C546,Jimmy Reed - The Sun Is Shining (Digitally ...,Slim Harpo - I Got Love If You Want It,TRMMREB12903CEB1B1
2,SOLPHZY12AC468ABA8,Africa HiTech - Footstep,Marcus Worgull - Drumstern (BONUS TRACK),TRMMBOC12903CEB46E
3,SONGHTM12A8C1374EF,Death in Vegas - Anita Berber,Valen Hsu - Shi Yi,TRMMITP128F425D8D0
4,SONGXCA12A8C13E82E,Grupo Exterminador - El Triunfador,I Ribelli - Lei M'Ama,TRMMAYZ128F429ECE6


## Part 5. Upload Known Song Duplicates

> #### <font color =red >_The purpose of uploading this dataset is to clean MSD later. Duplicate entries can affect model fitting later on._</font>

In [39]:
%time 

#Upload known song duplicates. More information of mismatch issue can be found here:
#https://labrosa.ee.columbia.edu/millionsong/blog/11-3-15-921810-song-dataset-duplicates

url = 'http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/msd_duplicates.txt'

urlretrieve(url, 'msd_duplicates.txt')

txt_file_dup = r'msd_duplicates.txt'

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


In [40]:
#Read mistmatches txt files as a dataframe. It reads all the data as one column

msd_duplicates_data = pd.read_csv(txt_file_dup,sep='delimeter',
                                 header=None, engine = 'python',names = ['duplicates'])


msd_duplicates_data.head(10)

,duplicates
0,# MILLION SONG DATASET - DUPLICATES
1,"# created by T. Bertin-Mahieux, Columbia Un..."
2,# tb2332@columbia.edu
3,# on Mon Mar 14 23:41:24 2011
4,# List of duplicates from artist names / artis...
5,# titles / song id.
6,%1 The Del Vikings - Whispering Bells
7,TRFCVSW12903D0A298
8,TRCWFEM128F9320F94
9,TRKYJRK12903CE6493


In [41]:
#Split song name and track id

msd_duplicates_data = msd_duplicates_data.duplicates.str.split('%',expand=True)

msd_duplicates_data.columns = ['track_id','song_name','blank']

msd_duplicates_data.info()

msd_duplicates_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185138 entries, 0 to 185137
Data columns (total 3 columns):
track_id     185138 non-null object
song_name    53471 non-null object
blank        7 non-null object
dtypes: object(3)
memory usage: 4.2+ MB


,track_id,song_name,blank
0,# MILLION SONG DATASET - DUPLICATES,None,None
1,"# created by T. Bertin-Mahieux, Columbia Un...",None,None
2,# tb2332@columbia.edu,None,None
3,# on Mon Mar 14 23:41:24 2011,None,None
4,# List of duplicates from artist names / artis...,None,None
5,# titles / song id.,None,None
6,,1 The Del Vikings - Whispering Bells,None
7,TRFCVSW12903D0A298,None,None
8,TRCWFEM128F9320F94,None,None
9,TRKYJRK12903CE6493,None,None


In [42]:
#drop blank column

msd_duplicates_data.drop('blank',axis =1).head()

,track_id,song_name
0,# MILLION SONG DATASET - DUPLICATES,None
1,"# created by T. Bertin-Mahieux, Columbia Un...",None
2,# tb2332@columbia.edu,None
3,# on Mon Mar 14 23:41:24 2011,None
4,# List of duplicates from artist names / artis...,None


In [43]:
#Replace blanks in track_id with np.nan

msd_duplicates_data['track_id'] = msd_duplicates_data['track_id'].replace('',np.nan)

msd_duplicates_data.head(10)

,track_id,song_name,blank
0,# MILLION SONG DATASET - DUPLICATES,None,None
1,"# created by T. Bertin-Mahieux, Columbia Un...",None,None
2,# tb2332@columbia.edu,None,None
3,# on Mon Mar 14 23:41:24 2011,None,None
4,# List of duplicates from artist names / artis...,None,None
5,# titles / song id.,None,None
6,NaN,1 The Del Vikings - Whispering Bells,None
7,TRFCVSW12903D0A298,None,None
8,TRCWFEM128F9320F94,None,None
9,TRKYJRK12903CE6493,None,None


In [44]:
#Drop first six lines. There are contact information.

msd_duplicates_data = msd_duplicates_data.drop(range(0,6))

msd_duplicates_data.head()

,track_id,song_name,blank
6,NaN,1 The Del Vikings - Whispering Bells,None
7,TRFCVSW12903D0A298,None,None
8,TRCWFEM128F9320F94,None,None
9,TRKYJRK12903CE6493,None,None
10,TRWTOBV128F9300F8A,None,None


In [45]:
#Fill missing song names using foward fill method. Missing entries correspond to the previous song name. 

msd_duplicates_data[['song_name']] = msd_duplicates_data[['song_name']].fillna(method='ffill')

# Remove rows with blank track id (these are blanks)

msd_duplicates_data = msd_duplicates_data[['track_id','song_name']].dropna().reset_index(drop=True)

msd_duplicates_data.head(10)

,track_id,song_name
0,TRFCVSW12903D0A298,1 The Del Vikings - Whispering Bells
1,TRCWFEM128F9320F94,1 The Del Vikings - Whispering Bells
2,TRKYJRK12903CE6493,1 The Del Vikings - Whispering Bells
3,TRWTOBV128F9300F8A,1 The Del Vikings - Whispering Bells
4,TRWFIGX128F42920CA,2 ANGELZOOM - Blasphemous rumours
5,TRFBNON128F4292174,2 ANGELZOOM - Blasphemous rumours
6,TRXXZSH128F1468B07,3 Dion & The Belmonts - A Lover's Prayer
7,TRHCDQC12903CB39D3,3 Dion & The Belmonts - A Lover's Prayer
8,TRUYEKK128F42591B4,4 Sentidos Opuestos - La Vida
9,TRVIIZQ128F425C2B8,4 Sentidos Opuestos - La Vida


In [46]:
# Remove rows with blank track id (these are blanks)

msd_duplicates_data['track_id'].dropna().reset_index(drop=True)

msd_duplicates_data.head(10)

,track_id,song_name
0,TRFCVSW12903D0A298,1 The Del Vikings - Whispering Bells
1,TRCWFEM128F9320F94,1 The Del Vikings - Whispering Bells
2,TRKYJRK12903CE6493,1 The Del Vikings - Whispering Bells
3,TRWTOBV128F9300F8A,1 The Del Vikings - Whispering Bells
4,TRWFIGX128F42920CA,2 ANGELZOOM - Blasphemous rumours
5,TRFBNON128F4292174,2 ANGELZOOM - Blasphemous rumours
6,TRXXZSH128F1468B07,3 Dion & The Belmonts - A Lover's Prayer
7,TRHCDQC12903CB39D3,3 Dion & The Belmonts - A Lover's Prayer
8,TRUYEKK128F42591B4,4 Sentidos Opuestos - La Vida
9,TRVIIZQ128F425C2B8,4 Sentidos Opuestos - La Vida


In [47]:
#Remove number in front of song names. If I dont it cannot properly identify duplicate sing entries.

msd_duplicates_data['song_name'] = msd_duplicates_data['song_name'].str.replace('\d+', '')

#See dataframe

msd_duplicates_data.head(10)

,track_id,song_name
0,TRFCVSW12903D0A298,The Del Vikings - Whispering Bells
1,TRCWFEM128F9320F94,The Del Vikings - Whispering Bells
2,TRKYJRK12903CE6493,The Del Vikings - Whispering Bells
3,TRWTOBV128F9300F8A,The Del Vikings - Whispering Bells
4,TRWFIGX128F42920CA,ANGELZOOM - Blasphemous rumours
5,TRFBNON128F4292174,ANGELZOOM - Blasphemous rumours
6,TRXXZSH128F1468B07,Dion & The Belmonts - A Lover's Prayer
7,TRHCDQC12903CB39D3,Dion & The Belmonts - A Lover's Prayer
8,TRUYEKK128F42591B4,Sentidos Opuestos - La Vida
9,TRVIIZQ128F425C2B8,Sentidos Opuestos - La Vida


In [48]:
#Determine unique and duplicate values false=unique, true=duplicate.

msd_duplicates_data['duplicate']= msd_duplicates_data.duplicated('song_name')

#Remove unique values from duplicate list. Leave duplicate values.

msd_duplicates_data = msd_duplicates_data[msd_duplicates_data['duplicate'] == True]

#See dataframe

msd_duplicates_data.head(10)

,track_id,song_name,duplicate
1,TRCWFEM128F9320F94,The Del Vikings - Whispering Bells,True
2,TRKYJRK12903CE6493,The Del Vikings - Whispering Bells,True
3,TRWTOBV128F9300F8A,The Del Vikings - Whispering Bells,True
5,TRFBNON128F4292174,ANGELZOOM - Blasphemous rumours,True
7,TRHCDQC12903CB39D3,Dion & The Belmonts - A Lover's Prayer,True
9,TRVIIZQ128F425C2B8,Sentidos Opuestos - La Vida,True
11,TRIBQPK128F92F06DE,George Hamilton IV - Abilene,True
12,TRKPICM12903CC84E6,George Hamilton IV - Abilene,True
13,TRSAVIQ128F428A577,George Hamilton IV - Abilene,True
14,TRWJBVG128F92FC109,George Hamilton IV - Abilene,True


## Part 6. Concatenate Datasets Song Metadata, Analysis Metadata & Musicbrainz Metadata to Create Song Aggregate File

Reminder: These were created by iteration so the index correspond to each song throughout the three metadata datasets.

In [49]:
#Concatenate metadata, analysis metadata and musicbrainz metadata on the column axis.

%time

#Concatenate metadata files.

aggregate_songs = pd.concat([song_metadata,
                             analysis_metadata,
                             musicbrainz_metadata], axis=1)

aggregate_songs.info()

aggregate_songs.head(10)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 53 columns):
analyzer_version                  1000000 non-null object
artist_7digitalid                 1000000 non-null int32
artist_familiarity                999815 non-null float64
artist_hotttnesss                 999988 non-null float64
artist_id                         1000000 non-null object
artist_latitude                   357492 non-null float64
artist_location                   1000000 non-null object
artist_longitude                  357492 non-null float64
artist_mbid                       1000000 non-null object
artist_name                       1000000 non-null object
artist_playmeid                   1000000 non-null int32
genre                             1000000 non-null object
idx_artist_terms                  1000000 non-null int32
idx_similar_artists               1000000 non-null int32
release     

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year
0,,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,...,-4.829,0,0.688,236.635,87.002,4,0.940,TRMMMYQ128F932D901,0,2003.0
1,,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,...,-10.555,1,0.355,148.660,150.778,1,0.000,TRMMMKD128F425225D,0,1995.0
2,,63531,0.643681,0.437504,ARGEKB01187FB50750,55.85780,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,...,-2.060,1,0.566,138.971,177.768,4,0.446,TRMMMRX128F93187D9,0,2006.0
3,,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,...,-4.654,1,0.451,138.687,87.433,4,0.000,TRMMMCH128F425532C,0,2003.0
4,,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,,NaN,,Der Mystic,...,-7.806,0,0.290,506.717,140.035,4,0.315,TRMMMWA128F426B589,0,NaN
5,,219281,0.361287,0.109626,AR2NS5Y1187FB5879D,NaN,,NaN,d087b377-bab7-46c4-bd12-15debebb5d61,David Montgomery,...,-21.420,1,0.581,811.799,90.689,4,0.158,TRMMMXN128F42936A5,0,NaN
6,,3736,0.692923,0.453732,ARO41T51187FB397AB,NaN,"Mexico City, Mexico",NaN,d2461c0a-5575-4425-a225-fce0180de3fd,Sasha / Turbulence,...,-4.931,0,0.627,206.629,101.450,1,0.960,TRMMMLR128F1494097,0,NaN
7,,49941,0.588156,0.401092,AR3Z9WY1187FB4CDC2,NaN,,NaN,f61e8ff-7621-4655-8ebd-68210645c5e9,Kris Kross,...,-12.214,0,0.350,212.120,98.020,4,0.982,TRMMMBB12903CB7D21,0,1993.0
8,,15202,0.408465,0.285901,ARA04401187B991E6E,54.99241,"Londonderry, Northern Ireland",-7.31923,1a9bf859-1dc2-495b-9e7c-289be7731a9f,Joseph Locke,...,-10.705,1,0.523,130.479,115.427,1,0.324,TRMMMHY12903CB53F1,0,NaN
9,,76721,0.419941,0.249137,ARCVMYS12454A51E6E,NaN,,NaN,,"""The Sun Harbor's Chorus-Documentary Recordings""",...,-20.160,0,0.480,104.489,124.339,4,1.000,TRMMMML128F4280EE9,0,NaN


## Part 7. Remove Song Mismatches from Aggregate Song File

In [50]:
#Filtering the song_id column did not work because they do not correspond exactly.

%time

#Merge song mistmaches and aggregate metadatafiles

aggregate_songs_mis = aggregate_songs.merge(msd_mismatch_data,how='left', on='song_id')

aggregate_songs_mis.info()
aggregate_songs_mis.head()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000378 entries, 0 to 1000377
Data columns (total 56 columns):
analyzer_version                  1000378 non-null object
artist_7digitalid                 1000378 non-null int32
artist_familiarity                1000193 non-null float64
artist_hotttnesss                 1000366 non-null float64
artist_id                         1000378 non-null object
artist_latitude                   357606 non-null float64
artist_location                   1000378 non-null object
artist_longitude                  357606 non-null float64
artist_mbid                       1000378 non-null object
artist_name                       1000378 non-null object
artist_playmeid                   1000378 non-null int32
genre                             1000378 non-null object
idx_artist_terms                  1000378 non-null int32
idx_similar_artists               1000378 non-null int32
release   

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id_x,idx_artist_mbtags,year,song_name,does_not_equal_to_song,track_id_y
0,,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,...,236.635,87.002,4,0.940,TRMMMYQ128F932D901,0,2003.0,NaN,NaN,NaN
1,,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,...,148.660,150.778,1,0.000,TRMMMKD128F425225D,0,1995.0,NaN,NaN,NaN
2,,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,...,138.971,177.768,4,0.446,TRMMMRX128F93187D9,0,2006.0,NaN,NaN,NaN
3,,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,...,138.687,87.433,4,0.000,TRMMMCH128F425532C,0,2003.0,NaN,NaN,NaN
4,,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,,NaN,,Der Mystic,...,506.717,140.035,4,0.315,TRMMMWA128F426B589,0,NaN,NaN,NaN,NaN


In [51]:
%time

#Track ids that did not merge means they are NOT MISMATCHED. Fill na to indicate this.

aggregate_songs_mis['track_id_y'] = aggregate_songs_mis['track_id_y'].fillna('not mismatched')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [52]:
%time

#Filter dataframe by 'not mistmached.

aggregate_songs_mis = aggregate_songs_mis[aggregate_songs_mis['track_id_y'] == 'not mismatched']

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


In [53]:
aggregate_songs_mis.head()

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id_x,idx_artist_mbtags,year,song_name,does_not_equal_to_song,track_id_y
0,,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,...,236.635,87.002,4,0.940,TRMMMYQ128F932D901,0,2003.0,NaN,NaN,not mismatched
1,,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,...,148.660,150.778,1,0.000,TRMMMKD128F425225D,0,1995.0,NaN,NaN,not mismatched
2,,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,...,138.971,177.768,4,0.446,TRMMMRX128F93187D9,0,2006.0,NaN,NaN,not mismatched
3,,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,...,138.687,87.433,4,0.000,TRMMMCH128F425532C,0,2003.0,NaN,NaN,not mismatched
4,,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,,NaN,,Der Mystic,...,506.717,140.035,4,0.315,TRMMMWA128F426B589,0,NaN,NaN,NaN,not mismatched


In [54]:
%time

#Rename track_id_x to track_id column

aggregate_songs_mis.rename(columns={'track_id_x':'track_id'}, inplace=True)

aggregate_songs_mis.info()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs
<class 'pandas.core.frame.DataFrame'>
Int64Index: 981022 entries, 0 to 1000377
Data columns (total 56 columns):
analyzer_version                  981022 non-null object
artist_7digitalid                 981022 non-null int32
artist_familiarity                980838 non-null float64
artist_hotttnesss                 981010 non-null float64
artist_id                         981022 non-null object
artist_latitude                   351971 non-null float64
artist_location                   981022 non-null object
artist_longitude                  351971 non-null float64
artist_mbid                       981022 non-null object
artist_name                       981022 non-null object
artist_playmeid                   981022 non-null int32
genre                             981022 non-null object
idx_artist_terms                  981022 non-null int32
idx_similar_artists               981022 non-null int32
release               

## Part 8. Remove Duplicates from Aggregate Song File

In [55]:
%time

aggregate_songs_mis_dup = aggregate_songs_mis.merge(msd_duplicates_data,
                                                      how='left',on='track_id')

aggregate_songs_mis_dup.info()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs
<class 'pandas.core.frame.DataFrame'>
Int64Index: 981022 entries, 0 to 981021
Data columns (total 58 columns):
analyzer_version                  981022 non-null object
artist_7digitalid                 981022 non-null int32
artist_familiarity                980838 non-null float64
artist_hotttnesss                 981010 non-null float64
artist_id                         981022 non-null object
artist_latitude                   351971 non-null float64
artist_location                   981022 non-null object
artist_longitude                  351971 non-null float64
artist_mbid                       981022 non-null object
artist_name                       981022 non-null object
artist_playmeid                   981022 non-null int32
genre                             981022 non-null object
idx_artist_terms                  981022 non-null int32
idx_similar_artists               981022 non-null int32
release                

In [56]:
%time

#Song names left blank after merge means they are NOT DUPLICATE. Fill na to indicate this.

aggregate_songs_mis_dup['song_name_y'] = aggregate_songs_mis_dup['song_name_y'].fillna('not_duplicate')


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [57]:
%time

#Filter dataframe by 'not duplicate.

aggregate_songs_mis_dup = aggregate_songs_mis_dup[aggregate_songs_mis_dup['song_name_y'] == 'not_duplicate']

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


In [58]:
#See dataframe

aggregate_songs_mis_dup.info()

aggregate_songs_mis_dup.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905712 entries, 0 to 981021
Data columns (total 58 columns):
analyzer_version                  905712 non-null object
artist_7digitalid                 905712 non-null int32
artist_familiarity                905531 non-null float64
artist_hotttnesss                 905700 non-null float64
artist_id                         905712 non-null object
artist_latitude                   325985 non-null float64
artist_location                   905712 non-null object
artist_longitude                  325985 non-null float64
artist_mbid                       905712 non-null object
artist_name                       905712 non-null object
artist_playmeid                   905712 non-null int32
genre                             905712 non-null object
idx_artist_terms                  905712 non-null int32
idx_similar_artists               905712 non-null int32
release                           905712 non-null object
release_7digitalid                

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year,song_name_x,does_not_equal_to_song,track_id_y,song_name_y,duplicate
0,,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,...,4,0.940,TRMMMYQ128F932D901,0,2003.0,NaN,NaN,not mismatched,not_duplicate,NaN
1,,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,...,1,0.000,TRMMMKD128F425225D,0,1995.0,NaN,NaN,not mismatched,not_duplicate,NaN
2,,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,...,4,0.446,TRMMMRX128F93187D9,0,2006.0,NaN,NaN,not mismatched,not_duplicate,NaN
3,,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,...,4,0.000,TRMMMCH128F425532C,0,2003.0,NaN,NaN,not mismatched,not_duplicate,NaN
4,,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,,NaN,,Der Mystic,...,4,0.315,TRMMMWA128F426B589,0,NaN,NaN,NaN,not mismatched,not_duplicate,NaN


## Part 9. Drop Columns Created in Merges & Fill Blanks with NaN from Song Metadata

In [59]:
#Drop columns created by merges

aggregate_songs_mis_dup = aggregate_songs_mis_dup.drop(['song_name_x','does_not_equal_to_song',
                                                        'track_id_y','song_name_y','duplicate',
                                                        'duplicate'],axis=1)

In [60]:
#Fill blanks with NaN

aggregate_songs_mis_dup = aggregate_songs_mis_dup.fillna(np.nan)

aggregate_songs_mis_dup = aggregate_songs_mis_dup.replace('',np.nan)

In [61]:
aggregate_songs_mis_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905712 entries, 0 to 981021
Data columns (total 53 columns):
analyzer_version                  0 non-null float64
artist_7digitalid                 905712 non-null int64
artist_familiarity                905531 non-null float64
artist_hotttnesss                 905700 non-null float64
artist_id                         905712 non-null object
artist_latitude                   325985 non-null float64
artist_location                   460185 non-null object
artist_longitude                  325985 non-null float64
artist_mbid                       847639 non-null object
artist_name                       905712 non-null object
artist_playmeid                   905712 non-null int64
genre                             0 non-null float64
idx_artist_terms                  905712 non-null int64
idx_similar_artists               905712 non-null int64
release                           905712 non-null object
release_7digitalid                905712 n

In [62]:
#drop empty columns

aggregate_songs_mis_dup = aggregate_songs_mis_dup.drop(['analyzer_version','genre',],axis=1)

aggregate_songs_mis_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905712 entries, 0 to 981021
Data columns (total 51 columns):
artist_7digitalid                 905712 non-null int64
artist_familiarity                905531 non-null float64
artist_hotttnesss                 905700 non-null float64
artist_id                         905712 non-null object
artist_latitude                   325985 non-null float64
artist_location                   460185 non-null object
artist_longitude                  325985 non-null float64
artist_mbid                       847639 non-null object
artist_name                       905712 non-null object
artist_playmeid                   905712 non-null int64
idx_artist_terms                  905712 non-null int64
idx_similar_artists               905712 non-null int64
release                           905712 non-null object
release_7digitalid                905712 non-null int64
song_hotttnesss                   551532 non-null float64
song_id                           

## Part 10. Convert Clean Song Metadata Dataset into Tidy Data Format

> #### <font color =red >_Melt User Data_</font>

In [63]:
users_listening_data.head()

,user_id,song_id,play_count,play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,1


In [64]:
%time

msd_user_data = pd.melt(frame=users_listening_data, id_vars='user_id',
        value_vars=['song_id', 'play_count','play'],
        var_name='listening_var', value_name='value')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


In [65]:
msd_user_data.head()

,user_id,listening_var,value
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,song_id,SOAKIMP12A8C130995
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,song_id,SOAPDEY12A81C210A9
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,song_id,SOBBMDR12A8C13253B
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,song_id,SOBFNSP12AF72A0E22
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,song_id,SOBFOVM12A58A7D494


> #### <font color =red >_Melt Song Metadata_</font>

In [66]:
#Create list of columns to melt

msd_songs_columns = list(aggregate_songs_mis_dup.columns)

msd_songs_columns.pop(15) #remove song_id from list. I do not want to melt song_id.

'song_id'

In [67]:
aggregate_songs_mis_dup.head()

,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,...,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year
0,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,NaN,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,44895,...,-4.829,0,0.688,236.635,87.002,4,0.940,TRMMMYQ128F932D901,0,2003.0
1,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,NaN,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,-1,...,-10.555,1,0.355,148.660,150.778,1,0.000,TRMMMKD128F425225D,0,1995.0
2,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,-1,...,-2.060,1,0.566,138.971,177.768,4,0.446,TRMMMRX128F93187D9,0,2006.0
3,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,NaN,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,34000,...,-4.654,1,0.451,138.687,87.433,4,0.000,TRMMMCH128F425532C,0,2003.0
4,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,NaN,NaN,NaN,Der Mystic,-1,...,-7.806,0,0.290,506.717,140.035,4,0.315,TRMMMWA128F426B589,0,NaN


In [68]:
%time 

msd_song_metadata = pd.melt(frame=aggregate_songs_mis_dup, id_vars='song_id',
                            value_vars = msd_songs_columns,
                            var_name='song_metadata', value_name='value')


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 24.1 µs


In [69]:
msd_song_metadata.head()

,song_id,song_metadata,value
0,SOQMMHC12AB0180CB8,artist_7digitalid,4069
1,SOVFVAK12A8C1350D9,artist_7digitalid,113480
2,SOGTUKN12AB017F4F1,artist_7digitalid,63531
3,SOBNYVR12A8C13558C,artist_7digitalid,65051
4,SOHSBXH12A8C13B0DF,artist_7digitalid,158279


> #### <font color =red >_Melt Tag and Term Metatada_</font>

In [70]:
artist_tags.head()

,artist_id,mbtag,terms
0,AR002UA1187B9A637D,uk,garage rock
1,AR002UA1187B9A637D,uk,country rock
2,AR002UA1187B9A637D,uk,free jazz
3,AR002UA1187B9A637D,uk,oi
4,AR002UA1187B9A637D,uk,space rock


In [71]:
msd_tags = pd.melt(frame=artist_tags, id_vars='artist_id',
        value_vars=['mbtag', 'terms'],
        var_name='tag', value_name='value')

In [72]:
msd_tags.head()

,artist_id,tag,value
0,AR002UA1187B9A637D,mbtag,uk
1,AR002UA1187B9A637D,mbtag,uk
2,AR002UA1187B9A637D,mbtag,uk
3,AR002UA1187B9A637D,mbtag,uk
4,AR002UA1187B9A637D,mbtag,uk


## Part 11. Output Datasets to CSV format

In [73]:
#User Data to CSV

users_listening_data.to_csv('msd_user_data.csv',chunksize = 500000, mode = 'a')

In [74]:
#Song Metada Data to CSV

aggregate_songs_mis_dup.to_csv('msd_tags.csv',chunksize = 500000, mode = 'a')

In [75]:
#Tags to CSV

%time

artist_tags.to_csv('msd_tags.csv', chunksize = 500000, mode = 'a')

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 68.9 µs


## Part 12. Output Tidy Format Datasets to CSV format

In [76]:
#User Data to CSV

%time

msd_user_data.to_csv('msd_user_data_tf.csv', chunksize = 500000, mode = 'a')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.78 µs


In [77]:
#Song data to CSV

%time

msd_song_metadata.to_csv('msd_song_metadata_tf.csv', chunksize = 500000, mode = 'a')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 31 µs


In [78]:
#Tags to CSV

%time

msd_tags.to_csv('msd_tags_tf.csv', chunksize = 500000, mode = 'a')

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 31.9 µs


# End of Notebook